<a href="https://colab.research.google.com/github/ShreyasJothish/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/DS43SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

### Neuron:

An artificial neuron is a mathematical function are elementary units in an artificial neural network. The artificial neuron receives one or more inputs and sums them to produce an output. Usually each input is separately weighted, and the sum is passed through a non-linear function known as an **activation  or transfer function**. The transfer functions can be sigmoid, rectified linear unit ReLU or step functions...

### Input Layer:

* The Input Layer is the 1st neuron layer which receives input from our dataset. 
* This is also called the visible layer because it's the only part that is exposed to our data and that our data interacts with directly. 
* Typically node maps within the input layer corresponds with one input node for each of the different inputs/features/columns of our dataset that will be passed to the neural network.

### Hidden Layer:

* The neuron layers between Input layer and Output layer are called Hidden layers since it cannot be accessed directly.
* Weighted sum of inputs and bias from previous layers are passed onto hidden layer for processing.
* There can be multiple hidden layers and usually refers to Deep Learning.

### Output Layer:

* The final layer of Neural Network is called Output layer.
* The purpose of the output layer is to output a vector of values that is in a format that is suitable for the type of problem that we're trying to address.
* This is handled using activation functions based on the problem.
1. Regression - Single output node for providing continous value.
2. Binary Classification - Sigmoid activation function in order to squishify values down to represent a probability.
3. Multi-class classification - Softmax function might have multiple output nodes in the output layer, one for each class that we're trying to predict.

### Activation Function:

In artificial neural networks, the activation function of a node defines the output of that node, or "neuron," given an input or set of inputs. This output is then used as input for the next node and so on until a desired solution to the original problem is found.

It maps the resulting values into the desired range such as between 0 to 1 or -1 to 1 etc. (depending upon the choice of activation function). For example, the use of the logistic activation function would map all inputs in the real number domain into the range of 0 to 1.

### Backpropagation

Backpropagation is a method used in artificial neural networks to calculate a gradient that is needed in the calculation of the weights to be used in the network. Backpropagation is shorthand for "the backward propagation of errors," since an error is computed at the output and distributed backwards throughout the network’s layers. It is commonly used to train deep neural networks.

Backpropagation is a generalization of the delta rule to multi-layered feedforward networks, made possible by using the chain rule to iteratively compute gradients for each layer.


## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [0]:
##### Your Code Here #####
import numpy as np
np.random.seed(1)

inputs = np.array([[1, 1, 1],
                   [1, 0, 1],
                   [0, 1, 1],
                   [0, 0, 1]])
# Expected output for AND
correct_outputs = [[1],
                  [0],
                  [0],
                  [0]]

# Initial weights centered around 0
weights = 2 * np.random.random((3,1)) - 1

In [0]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
  return sigmoid(x) * (1 - sigmoid(x))

In [3]:
for iteration in range(10000):
  
  # Weighted sum of inputs and weights
  weighted_sum = np.dot(inputs, weights)
  
  # Activate with sigmoid function
  activated_output = sigmoid(weighted_sum)
  
  # Calculate Error
  error = correct_outputs - activated_output
  
  # Calculate weight adjustments with sigmoid_derivative
  adjustments = error * sigmoid_derivative(activated_output)
  
  # Update weights
  weights += np.dot(inputs.T, adjustments)
  
print('Optimized weights after training: ')
print(weights)

print("Output After Training:")
print(activated_output)

print("Final error:")
print(correct_outputs - activated_output)

Optimized weights after training: 
[[ 11.84079538]
 [ 11.84079538]
 [-18.04950231]]
Output After Training:
[[9.96431333e-01]
 [2.00799905e-03]
 [2.00799905e-03]
 [1.44987830e-08]]
Final error:
[[ 3.56866716e-03]
 [-2.00799905e-03]
 [-2.00799905e-03]
 [-1.44987830e-08]]


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [4]:
from google.colab import files
files.upload()

{}

In [5]:
##### Your Code Here #####
import numpy as np
import pandas as pd

df = pd.read_csv("heart.csv")
print(f'Shape:\n{df.shape}')
print(f'Checking for NaN:\n{df.isnull().sum()}')
print(f'Checking the type of columns')
for column in df.columns:
  print(f'{column},{df[column].dtype}')

Shape:
(303, 14)
Checking for NaN:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64
Checking the type of columns
age,int64
sex,int64
cp,int64
trestbps,int64
chol,int64
fbs,int64
restecg,int64
thalach,int64
exang,int64
oldpeak,float64
slope,int64
ca,int64
thal,int64
target,int64


In [0]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
				if (minmax[i][1] != minmax[i][0]):
						row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [7]:
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'heart.csv'
dataset = load_csv(filename)

# removing the header
print(f'Length of dataset with header {len(dataset)}')
print(f'dataset header being removed\n {dataset[0]}')
dataset.pop(0)
print(f'Length of dataset without header {len(dataset)}')

for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 3
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Length of dataset with header 304
dataset header being removed
 ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
Length of dataset without header 303
Scores: [86.66666666666667, 83.33333333333334, 83.33333333333334, 75.0, 88.33333333333333]
Mean Accuracy: 83.333%


## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [0]:
##### Your Code Here #####
# generic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv("heart.csv")

# Use the values to get np.arrary instead of passing the dataframe
X = df.drop(columns='target').values
y = df['target'].values
X.shape

(303, 13)

In [10]:
# Imports for pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV

# Keras imports
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [12]:
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=13, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', 
                metrics=['accuracy'])
	return model

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/1
242/242 [==============================] - 0s 28us/step
[CV] ....................... , score=0.4262295081967213, total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


Epoch 1/1
242/242 [==============================] - 0s 30us/step
[CV] ....................... , score=0.2786885245901639, total=   0.9s
[CV]  ................................................................


...

Epoch 1/1
243/243 [==============================] - 0s 27us/step
[CV] ..................................... , score=0.65, total=   1.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.0s remaining:    0.0s


Epoch 1/1
243/243 [==============================] - 0s 38us/step
[CV] ..................................... , score=0.95, total=   1.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.0s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/1
303/303 [==============================] - 1s 2ms/step - loss: 0.7592 - acc: 0.5116
Cross Validation Score: 0.5511551155115512
Best Parameters: {}


**Without Hyperparameter Tuning:**

Cross Validation Score: 0.5511551155115512

Best Parameters: {}

In [16]:
# Tunning Batch Size

# fix random seed for reproducibility
np.random.seed(42)

# Function to create model, required for KerasClassifier
def create_model():
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=13, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', \
                metrics=['accuracy'])
  return model

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [20, 40, 80, 100],
    'kerasclassifier__epochs': [50],
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] kerasclassifier__batch_size=20, kerasclassifier__epochs=50 ......


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
202/202 [==============================] - 2s 10ms/step - loss: 0.8338 - acc: 0.4257
Epoch 2/50
202/202 [==============================] - 0s 110us/step - loss: 0.7962 - acc: 0.5050
Epoch 3/50
202/202 [==============================] - 0s 98us/step - loss: 0.7643 - acc: 0.5099
Epoch 4/50
202/202 [==============================] - 0s 108us/step - loss: 0.7348 - acc: 0.5495
Epoch 5/50
202/202 [==============================] - 0s 103us/step - loss: 0.7085 - acc: 0.5891
Epoch 6/50
202/202 [==============================] - 0s 98us/step - loss: 0.6847 - acc: 0.6139
Epoch 7/50
202/202 [==============================] - 0s 93us/step - loss: 0.6632 - acc: 0.6287
Epoch 8/50
202/202 [==============================] - 0s 99us/step - loss: 0.6424 - acc: 0.6485
Epoch 9/50
202/202 [==============================] - 0s 98us/step - loss: 0.6223 - acc: 0.6634
Epoch 10/50
202/202 [==============================] - 0s 102us/step - loss: 0.6040 - acc: 0.6931
Epoch 11/50
202/202 [==============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 2s 9ms/step - loss: 0.6970 - acc: 0.5050
Epoch 2/50
202/202 [==============================] - 0s 91us/step - loss: 0.6788 - acc: 0.5297
Epoch 3/50
202/202 [==============================] - 0s 90us/step - loss: 0.6615 - acc: 0.5743
Epoch 4/50
202/202 [==============================] - 0s 92us/step - loss: 0.6459 - acc: 0.6139
Epoch 5/50
202/202 [==============================] - 0s 94us/step - loss: 0.6325 - acc: 0.6436
Epoch 6/50
202/202 [==============================] - 0s 90us/step - loss: 0.6195 - acc: 0.6733
Epoch 7/50
202/202 [==============================] - 0s 91us/step - loss: 0.6071 - acc: 0.6733
Epoch 8/50
202/202 [==============================] - 0s 92us/step - loss: 0.5945 - acc: 0.6980
Epoch 9/50
202/202 [==============================] - 0s 91us/step - loss: 0.5823 - acc: 0.7228
Epoch 10/50
202/202 [==============================] - 0s 93us/step - loss: 0.5701 - acc: 0.7376
Epoch 11/50
202/202 [===================

...

Epoch 1/50
202/202 [==============================] - 2s 11ms/step - loss: 0.6523 - acc: 0.7574
Epoch 2/50
202/202 [==============================] - 0s 58us/step - loss: 0.6458 - acc: 0.7772
Epoch 3/50
202/202 [==============================] - 0s 36us/step - loss: 0.6393 - acc: 0.8020
Epoch 4/50
202/202 [==============================] - 0s 56us/step - loss: 0.6323 - acc: 0.8020
Epoch 5/50
202/202 [==============================] - 0s 51us/step - loss: 0.6257 - acc: 0.8020
Epoch 6/50
202/202 [==============================] - 0s 62us/step - loss: 0.6192 - acc: 0.8119
Epoch 7/50
202/202 [==============================] - 0s 46us/step - loss: 0.6125 - acc: 0.8119
Epoch 8/50
202/202 [==============================] - 0s 38us/step - loss: 0.6060 - acc: 0.8168
Epoch 9/50
202/202 [==============================] - 0s 47us/step - loss: 0.6000 - acc: 0.8168
Epoch 10/50
202/202 [==============================] - 0s 54us/step - loss: 0.5938 - acc: 0.8168
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   36.0s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 2s 11ms/step - loss: 0.6463 - acc: 0.6188
Epoch 2/50
202/202 [==============================] - 0s 43us/step - loss: 0.6385 - acc: 0.6188
Epoch 3/50
202/202 [==============================] - 0s 39us/step - loss: 0.6310 - acc: 0.6188
Epoch 4/50
202/202 [==============================] - 0s 37us/step - loss: 0.6237 - acc: 0.6188
Epoch 5/50
202/202 [==============================] - 0s 40us/step - loss: 0.6173 - acc: 0.6238
Epoch 6/50
202/202 [==============================] - 0s 38us/step - loss: 0.6119 - acc: 0.6188
Epoch 7/50
202/202 [==============================] - 0s 39us/step - loss: 0.6062 - acc: 0.6139
Epoch 8/50
202/202 [==============================] - 0s 40us/step - loss: 0.6010 - acc: 0.6287
Epoch 9/50
202/202 [==============================] - 0s 44us/step - loss: 0.5956 - acc: 0.6238
Epoch 10/50
202/202 [==============================] - 0s 57us/step - loss: 0.5904 - acc: 0.6386
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   48.5s finished


Epoch 1/50
303/303 [==============================] - 2s 8ms/step - loss: 0.6518 - acc: 0.6073
Epoch 2/50
303/303 [==============================] - 0s 57us/step - loss: 0.6357 - acc: 0.6271
Epoch 3/50
303/303 [==============================] - 0s 54us/step - loss: 0.6206 - acc: 0.6634
Epoch 4/50
303/303 [==============================] - 0s 52us/step - loss: 0.6061 - acc: 0.6832
Epoch 5/50
303/303 [==============================] - 0s 55us/step - loss: 0.5926 - acc: 0.7030
Epoch 6/50
303/303 [==============================] - 0s 54us/step - loss: 0.5796 - acc: 0.7129
Epoch 7/50
303/303 [==============================] - 0s 54us/step - loss: 0.5669 - acc: 0.7294
Epoch 8/50
303/303 [==============================] - 0s 57us/step - loss: 0.5556 - acc: 0.7459
Epoch 9/50
303/303 [==============================] - 0s 54us/step - loss: 0.5447 - acc: 0.7558
Epoch 10/50
303/303 [==============================] - 0s 51us/step - loss: 0.5341 - acc: 0.7657
Epoch 11/50
303/303 [===================

**Hyperparameter tuning 1**

param_grid = {

    'kerasclassifier__batch_size': [20, 40, 80, 100],
    'kerasclassifier__epochs': [50],
}

Output:

Cross Validation Score: 0.5973597359735974

Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50}

In [17]:
# Tunning Optimizer

# fix random seed for reproducibility
np.random.seed(42)

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=13, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  model.compile(loss='binary_crossentropy', optimizer=optimizer, \
                metrics=['accuracy'])
  return model

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__optimizer': ['sgd', 'rmsprop', 'adagrad', 'adadelta', 
                                   'adam', 'adamax', 'nadam'],
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 7 candidates, totalling 21 fits
[CV] kerasclassifier__batch_size=40, kerasclassifier__epochs=50, kerasclassifier__optimizer=sgd 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
202/202 [==============================] - 2s 12ms/step - loss: 0.8403 - acc: 0.4257
Epoch 2/50
202/202 [==============================] - 0s 69us/step - loss: 0.8188 - acc: 0.4653
Epoch 3/50
202/202 [==============================] - 0s 68us/step - loss: 0.8040 - acc: 0.4901
Epoch 4/50
202/202 [==============================] - 0s 73us/step - loss: 0.7857 - acc: 0.5000
Epoch 5/50
202/202 [==============================] - 0s 69us/step - loss: 0.7721 - acc: 0.5198
Epoch 6/50
202/202 [==============================] - 0s 69us/step - loss: 0.7585 - acc: 0.5347
Epoch 7/50
202/202 [==============================] - 0s 71us/step - loss: 0.7455 - acc: 0.5446
Epoch 8/50
202/202 [==============================] - 0s 72us/step - loss: 0.7298 - acc: 0.5594
Epoch 9/50
202/202 [==============================] - 0s 68us/step - loss: 0.7189 - acc: 0.5941
Epoch 10/50
202/202 [==============================] - 0s 67us/step - loss: 0.7096 - acc: 0.5941
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 3s 13ms/step - loss: 0.7024 - acc: 0.4802
Epoch 2/50
202/202 [==============================] - 0s 75us/step - loss: 0.6980 - acc: 0.4901
Epoch 3/50
202/202 [==============================] - 0s 70us/step - loss: 0.6932 - acc: 0.5149
Epoch 4/50
202/202 [==============================] - 0s 66us/step - loss: 0.6886 - acc: 0.5198
Epoch 5/50
202/202 [==============================] - 0s 72us/step - loss: 0.6840 - acc: 0.5297
Epoch 6/50
202/202 [==============================] - 0s 79us/step - loss: 0.6807 - acc: 0.5347
Epoch 7/50
202/202 [==============================] - 0s 80us/step - loss: 0.6759 - acc: 0.5446
Epoch 8/50
202/202 [==============================] - 0s 68us/step - loss: 0.6721 - acc: 0.5446
Epoch 9/50
202/202 [==============================] - 0s 68us/step - loss: 0.6672 - acc: 0.5594
Epoch 10/50
202/202 [==============================] - 0s 67us/step - loss: 0.6641 - acc: 0.5743
Epoch 11/50
202/202 [==================

...

Epoch 1/50
202/202 [==============================] - 3s 14ms/step - loss: 0.6240 - acc: 0.7921
Epoch 2/50
202/202 [==============================] - 0s 65us/step - loss: 0.5797 - acc: 0.8168
Epoch 3/50
202/202 [==============================] - 0s 68us/step - loss: 0.5575 - acc: 0.8168
Epoch 4/50
202/202 [==============================] - 0s 67us/step - loss: 0.5413 - acc: 0.8168
Epoch 5/50
202/202 [==============================] - 0s 67us/step - loss: 0.5275 - acc: 0.8168
Epoch 6/50
202/202 [==============================] - 0s 71us/step - loss: 0.5154 - acc: 0.8168
Epoch 7/50
202/202 [==============================] - 0s 68us/step - loss: 0.5037 - acc: 0.8168
Epoch 8/50
202/202 [==============================] - 0s 71us/step - loss: 0.4920 - acc: 0.8168
Epoch 9/50
202/202 [==============================] - 0s 65us/step - loss: 0.4828 - acc: 0.8168
Epoch 10/50
202/202 [==============================] - 0s 68us/step - loss: 0.4746 - acc: 0.8168
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   43.0s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 3s 14ms/step - loss: 0.6410 - acc: 0.6188
Epoch 2/50
202/202 [==============================] - 0s 74us/step - loss: 0.6217 - acc: 0.6238
Epoch 3/50
202/202 [==============================] - 0s 71us/step - loss: 0.6039 - acc: 0.6139
Epoch 4/50
202/202 [==============================] - 0s 69us/step - loss: 0.5892 - acc: 0.6287
Epoch 5/50
202/202 [==============================] - 0s 70us/step - loss: 0.5782 - acc: 0.6535
Epoch 6/50
202/202 [==============================] - 0s 73us/step - loss: 0.5643 - acc: 0.6881
Epoch 7/50
202/202 [==============================] - 0s 72us/step - loss: 0.5533 - acc: 0.6931
Epoch 8/50
202/202 [==============================] - 0s 71us/step - loss: 0.5443 - acc: 0.7030
Epoch 9/50
202/202 [==============================] - 0s 71us/step - loss: 0.5357 - acc: 0.6980
Epoch 10/50
202/202 [==============================] - 0s 71us/step - loss: 0.5272 - acc: 0.7228
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:  1.8min finished


Epoch 1/50
303/303 [==============================] - 3s 11ms/step - loss: 0.7582 - acc: 0.4719
Epoch 2/50
303/303 [==============================] - 0s 58us/step - loss: 0.7252 - acc: 0.4818
Epoch 3/50
303/303 [==============================] - 0s 57us/step - loss: 0.6946 - acc: 0.5116
Epoch 4/50
303/303 [==============================] - 0s 58us/step - loss: 0.6656 - acc: 0.5875
Epoch 5/50
303/303 [==============================] - 0s 59us/step - loss: 0.6392 - acc: 0.6535
Epoch 6/50
303/303 [==============================] - 0s 57us/step - loss: 0.6129 - acc: 0.7129
Epoch 7/50
303/303 [==============================] - 0s 59us/step - loss: 0.5875 - acc: 0.7525
Epoch 8/50
303/303 [==============================] - 0s 58us/step - loss: 0.5625 - acc: 0.7855
Epoch 9/50
303/303 [==============================] - 0s 58us/step - loss: 0.5390 - acc: 0.7954
Epoch 10/50
303/303 [==============================] - 0s 56us/step - loss: 0.5168 - acc: 0.8086
Epoch 11/50
303/303 [==================

**Hyperparameter tuning 2**

param_grid = {

    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__optimizer': ['sgd', 'rmsprop', 'adagrad', 'adadelta', 
                                   'adam', 'adamax', 'nadam'],
}


Output:

Cross Validation Score: 0.6501650165016502

Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50, 'kerasclassifier__optimizer': 'nadam'}

In [18]:
# Tunning learning rate and schedule decay of optimizer
from keras.optimizers import Nadam

# fix random seed for reproducibility
np.random.seed(42)

# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.002, schedule_decay=0.004):
  # create model
  model = Sequential()
  model.add(Dense(12, input_dim=13, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  optimizer = Nadam(lr=learning_rate, schedule_decay=schedule_decay)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, \
                metrics=['accuracy'])
  return model

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__learning_rate': [0.001, 0.002, 0.004, 0.01, 0.1],
    'kerasclassifier__schedule_decay': [0.001, 0.002, 0.004, 0.01, 0.1],
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] kerasclassifier__batch_size=40, kerasclassifier__epochs=50, kerasclassifier__learning_rate=0.001, kerasclassifier__schedule_decay=0.001 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
202/202 [==============================] - 4s 18ms/step - loss: 0.8386 - acc: 0.4208
Epoch 2/50
202/202 [==============================] - 0s 70us/step - loss: 0.8190 - acc: 0.4604
Epoch 3/50
202/202 [==============================] - 0s 71us/step - loss: 0.8022 - acc: 0.4901
Epoch 4/50
202/202 [==============================] - 0s 75us/step - loss: 0.7838 - acc: 0.5149
Epoch 5/50
202/202 [==============================] - 0s 82us/step - loss: 0.7676 - acc: 0.5149
Epoch 6/50
202/202 [==============================] - 0s 74us/step - loss: 0.7517 - acc: 0.5248
Epoch 7/50
202/202 [==============================] - 0s 75us/step - loss: 0.7366 - acc: 0.5495
Epoch 8/50
202/202 [==============================] - 0s 77us/step - loss: 0.7199 - acc: 0.5693
Epoch 9/50
202/202 [==============================] - 0s 78us/step - loss: 0.7056 - acc: 0.5891
Epoch 10/50
202/202 [==============================] - 0s 76us/step - loss: 0.6929 - acc: 0.6089
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.4s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 3s 17ms/step - loss: 0.6994 - acc: 0.5050
Epoch 2/50
202/202 [==============================] - 0s 75us/step - loss: 0.6899 - acc: 0.5099
Epoch 3/50
202/202 [==============================] - 0s 76us/step - loss: 0.6804 - acc: 0.5347
Epoch 4/50
202/202 [==============================] - 0s 71us/step - loss: 0.6712 - acc: 0.5545
Epoch 5/50
202/202 [==============================] - 0s 70us/step - loss: 0.6627 - acc: 0.5743
Epoch 6/50
202/202 [==============================] - 0s 74us/step - loss: 0.6554 - acc: 0.5891
Epoch 7/50
202/202 [==============================] - 0s 80us/step - loss: 0.6473 - acc: 0.6238
Epoch 8/50
202/202 [==============================] - 0s 86us/step - loss: 0.6401 - acc: 0.6337
Epoch 9/50
202/202 [==============================] - 0s 75us/step - loss: 0.6324 - acc: 0.6535
Epoch 10/50
202/202 [==============================] - 0s 73us/step - loss: 0.6256 - acc: 0.6535
Epoch 11/50
202/202 [==================

...

Epoch 1/50
202/202 [==============================] - 4s 19ms/step - loss: 0.6491 - acc: 0.7624
Epoch 2/50
202/202 [==============================] - 0s 85us/step - loss: 0.6400 - acc: 0.7970
Epoch 3/50
202/202 [==============================] - 0s 78us/step - loss: 0.6321 - acc: 0.8020
Epoch 4/50
202/202 [==============================] - 0s 82us/step - loss: 0.6241 - acc: 0.8020
Epoch 5/50
202/202 [==============================] - 0s 78us/step - loss: 0.6160 - acc: 0.8069
Epoch 6/50
202/202 [==============================] - 0s 77us/step - loss: 0.6078 - acc: 0.8119
Epoch 7/50
202/202 [==============================] - 0s 85us/step - loss: 0.5991 - acc: 0.8168
Epoch 8/50
202/202 [==============================] - 0s 91us/step - loss: 0.5899 - acc: 0.8168
Epoch 9/50
202/202 [==============================] - 0s 79us/step - loss: 0.5821 - acc: 0.8168
Epoch 10/50
202/202 [==============================] - 0s 86us/step - loss: 0.5744 - acc: 0.8168
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   58.2s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 4s 20ms/step - loss: 0.6421 - acc: 0.6188
Epoch 2/50
202/202 [==============================] - 0s 76us/step - loss: 0.6284 - acc: 0.6238
Epoch 3/50
202/202 [==============================] - 0s 69us/step - loss: 0.6151 - acc: 0.6188
Epoch 4/50
202/202 [==============================] - 0s 74us/step - loss: 0.6029 - acc: 0.6188
Epoch 5/50
202/202 [==============================] - 0s 76us/step - loss: 0.5926 - acc: 0.6337
Epoch 6/50
202/202 [==============================] - 0s 75us/step - loss: 0.5812 - acc: 0.6386
Epoch 7/50
202/202 [==============================] - 0s 70us/step - loss: 0.5709 - acc: 0.6683
Epoch 8/50
202/202 [==============================] - 0s 70us/step - loss: 0.5625 - acc: 0.6782
Epoch 9/50
202/202 [==============================] - 0s 69us/step - loss: 0.5537 - acc: 0.6881
Epoch 10/50
202/202 [==============================] - 0s 70us/step - loss: 0.5455 - acc: 0.6980
Epoch 11/50
202/202 [==================

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 11.8min finished


Epoch 1/50
303/303 [==============================] - 8s 28ms/step - loss: 0.4777 - acc: 0.7591
Epoch 2/50
303/303 [==============================] - 0s 82us/step - loss: 0.3873 - acc: 0.8482
Epoch 3/50
303/303 [==============================] - 0s 83us/step - loss: 0.4005 - acc: 0.8251
Epoch 4/50
303/303 [==============================] - 0s 86us/step - loss: 0.3659 - acc: 0.8416
Epoch 5/50
303/303 [==============================] - 0s 82us/step - loss: 0.3321 - acc: 0.8713
Epoch 6/50
303/303 [==============================] - 0s 86us/step - loss: 0.3285 - acc: 0.8581
Epoch 7/50
303/303 [==============================] - 0s 75us/step - loss: 0.3097 - acc: 0.8779
Epoch 8/50
303/303 [==============================] - 0s 77us/step - loss: 0.3136 - acc: 0.8779
Epoch 9/50
303/303 [==============================] - 0s 80us/step - loss: 0.3870 - acc: 0.8416
Epoch 10/50
303/303 [==============================] - 0s 80us/step - loss: 0.2811 - acc: 0.8977
Epoch 11/50
303/303 [==================

**Hyperparameter tuning 3 and 4**

param_grid = {

    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__learning_rate': [0.001, 0.002, 0.004, 0.01, 0.1],
    'kerasclassifier__schedule_decay': [0.001, 0.002, 0.004, 0.01, 0.1],
}

Output:

Cross Validation Score: 0.7062706270627063

Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50, 'kerasclassifier__learning_rate': 0.1, 'kerasclassifier__schedule_decay': 0.01}

In [19]:
# Tunning number of nodes
from keras.optimizers import Nadam

# fix random seed for reproducibility
np.random.seed(42)

# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.002, schedule_decay=0.004,
                nodes=12):
  # create model
  model = Sequential()
  model.add(Dense(nodes, input_dim=13, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  optimizer = Nadam(lr=learning_rate, schedule_decay=schedule_decay)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, \
                metrics=['accuracy'])
  return model

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__learning_rate': [0.1],
    'kerasclassifier__schedule_decay': [0.01],
    'kerasclassifier__nodes': [4, 8, 16, 32, 64]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] kerasclassifier__batch_size=40, kerasclassifier__epochs=50, kerasclassifier__learning_rate=0.1, kerasclassifier__nodes=4, kerasclassifier__schedule_decay=0.01 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
202/202 [==============================] - 9s 42ms/step - loss: 0.6091 - acc: 0.5743
Epoch 2/50
202/202 [==============================] - 0s 93us/step - loss: 0.4595 - acc: 0.6832
Epoch 3/50
202/202 [==============================] - 0s 96us/step - loss: 0.4048 - acc: 0.8515
Epoch 4/50
202/202 [==============================] - 0s 95us/step - loss: 0.3900 - acc: 0.8416
Epoch 5/50
202/202 [==============================] - 0s 100us/step - loss: 0.4058 - acc: 0.7970
Epoch 6/50
202/202 [==============================] - 0s 95us/step - loss: 0.3695 - acc: 0.8366
Epoch 7/50
202/202 [==============================] - 0s 93us/step - loss: 0.3579 - acc: 0.8564
Epoch 8/50
202/202 [==============================] - 0s 93us/step - loss: 0.3425 - acc: 0.8663
Epoch 9/50
202/202 [==============================] - 0s 94us/step - loss: 0.3486 - acc: 0.8515
Epoch 10/50
202/202 [==============================] - 0s 104us/step - loss: 0.3447 - acc: 0.8614
Epoch 11/50
202/202 [================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 8s 40ms/step - loss: 0.5830 - acc: 0.7178
Epoch 2/50
202/202 [==============================] - 0s 100us/step - loss: 0.3594 - acc: 0.8515
Epoch 3/50
202/202 [==============================] - 0s 100us/step - loss: 0.4342 - acc: 0.8020
Epoch 4/50
202/202 [==============================] - 0s 99us/step - loss: 0.3494 - acc: 0.8614
Epoch 5/50
202/202 [==============================] - 0s 97us/step - loss: 0.3368 - acc: 0.8614
Epoch 6/50
202/202 [==============================] - 0s 105us/step - loss: 0.3374 - acc: 0.8515
Epoch 7/50
202/202 [==============================] - 0s 96us/step - loss: 0.3214 - acc: 0.8515
Epoch 8/50
202/202 [==============================] - 0s 94us/step - loss: 0.3126 - acc: 0.8713
Epoch 9/50
202/202 [==============================] - 0s 109us/step - loss: 0.3079 - acc: 0.8762
Epoch 10/50
202/202 [==============================] - 0s 92us/step - loss: 0.3297 - acc: 0.8713
Epoch 11/50
202/202 [==============

...

Epoch 1/50
202/202 [==============================] - 8s 42ms/step - loss: 0.4083 - acc: 0.8465
Epoch 2/50
202/202 [==============================] - 0s 97us/step - loss: 0.3626 - acc: 0.8515
Epoch 3/50
202/202 [==============================] - 0s 92us/step - loss: 0.2588 - acc: 0.8911
Epoch 4/50
202/202 [==============================] - 0s 99us/step - loss: 0.2294 - acc: 0.9208
Epoch 5/50
202/202 [==============================] - 0s 98us/step - loss: 0.6203 - acc: 0.8812
Epoch 6/50
202/202 [==============================] - 0s 95us/step - loss: 0.2459 - acc: 0.9109
Epoch 7/50
202/202 [==============================] - 0s 100us/step - loss: 0.1881 - acc: 0.9406
Epoch 8/50
202/202 [==============================] - 0s 102us/step - loss: 0.1928 - acc: 0.9356
Epoch 9/50
202/202 [==============================] - 0s 92us/step - loss: 0.4776 - acc: 0.8614
Epoch 10/50
202/202 [==============================] - 0s 100us/step - loss: 0.3466 - acc: 0.8762
Epoch 11/50
202/202 [===============

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.0min remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 8s 42ms/step - loss: 0.5664 - acc: 0.6782
Epoch 2/50
202/202 [==============================] - 0s 100us/step - loss: 0.3578 - acc: 0.8416
Epoch 3/50
202/202 [==============================] - 0s 97us/step - loss: 0.4929 - acc: 0.8069
Epoch 4/50
202/202 [==============================] - 0s 99us/step - loss: 0.6004 - acc: 0.8069
Epoch 5/50
202/202 [==============================] - 0s 98us/step - loss: 0.7936 - acc: 0.8218
Epoch 6/50
202/202 [==============================] - 0s 93us/step - loss: 0.3762 - acc: 0.8465
Epoch 7/50
202/202 [==============================] - 0s 94us/step - loss: 0.6098 - acc: 0.8416
Epoch 8/50
202/202 [==============================] - 0s 96us/step - loss: 0.4288 - acc: 0.8564
Epoch 9/50
202/202 [==============================] - 0s 97us/step - loss: 1.0658 - acc: 0.7277
Epoch 10/50
202/202 [==============================] - 0s 95us/step - loss: 0.4083 - acc: 0.8663
Epoch 11/50
202/202 [=================

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.4min finished


Epoch 1/50
303/303 [==============================] - 9s 29ms/step - loss: 0.4622 - acc: 0.7756
Epoch 2/50
303/303 [==============================] - 0s 83us/step - loss: 0.3846 - acc: 0.8383
Epoch 3/50
303/303 [==============================] - 0s 84us/step - loss: 0.3723 - acc: 0.8350
Epoch 4/50
303/303 [==============================] - 0s 85us/step - loss: 0.3695 - acc: 0.8350
Epoch 5/50
303/303 [==============================] - 0s 90us/step - loss: 0.3533 - acc: 0.8416
Epoch 6/50
303/303 [==============================] - 0s 84us/step - loss: 0.3445 - acc: 0.8680
Epoch 7/50
303/303 [==============================] - 0s 91us/step - loss: 0.3305 - acc: 0.8614
Epoch 8/50
303/303 [==============================] - 0s 88us/step - loss: 0.3424 - acc: 0.8449
Epoch 9/50
303/303 [==============================] - 0s 84us/step - loss: 0.3195 - acc: 0.8713
Epoch 10/50
303/303 [==============================] - 0s 83us/step - loss: 0.3920 - acc: 0.8449
Epoch 11/50
303/303 [==================

**Hyperparameter tuning 5:**

param_grid = {

    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__learning_rate': [0.1],
    'kerasclassifier__schedule_decay': [0.01],
    'kerasclassifier__nodes': [4, 8, 16, 32, 64]
}

Output:

Cross Validation Score: 0.735973597359736

Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50, 'kerasclassifier__learning_rate': 0.1, 'kerasclassifier__nodes': 8, 'kerasclassifier__schedule_decay': 0.01}

In [20]:
# Finally tuning number of epochs

# Tunning number of nodes
from keras.optimizers import Nadam

# fix random seed for reproducibility
np.random.seed(42)

# Function to create model, required for KerasClassifier
def create_model(learning_rate=0.002, schedule_decay=0.004,
                nodes=12):
  # create model
  model = Sequential()
  model.add(Dense(nodes, input_dim=13, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model
  optimizer = Nadam(lr=learning_rate, schedule_decay=schedule_decay)
  model.compile(loss='binary_crossentropy', optimizer=optimizer, \
                metrics=['accuracy'])
  return model

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50, 100, 150, 200],
    'kerasclassifier__learning_rate': [0.1],
    'kerasclassifier__schedule_decay': [0.01],
    'kerasclassifier__nodes': [8]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] kerasclassifier__batch_size=40, kerasclassifier__epochs=50, kerasclassifier__learning_rate=0.1, kerasclassifier__nodes=8, kerasclassifier__schedule_decay=0.01 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/50
202/202 [==============================] - 9s 45ms/step - loss: 0.6403 - acc: 0.6535
Epoch 2/50
202/202 [==============================] - 0s 95us/step - loss: 0.3696 - acc: 0.8267
Epoch 3/50
202/202 [==============================] - 0s 105us/step - loss: 0.3746 - acc: 0.8515
Epoch 4/50
202/202 [==============================] - 0s 102us/step - loss: 0.3463 - acc: 0.8465
Epoch 5/50
202/202 [==============================] - 0s 102us/step - loss: 0.4204 - acc: 0.7871
Epoch 6/50
202/202 [==============================] - 0s 97us/step - loss: 0.3576 - acc: 0.8366
Epoch 7/50
202/202 [==============================] - 0s 98us/step - loss: 0.3198 - acc: 0.8465
Epoch 8/50
202/202 [==============================] - 0s 99us/step - loss: 0.3173 - acc: 0.8515
Epoch 9/50
202/202 [==============================] - 0s 102us/step - loss: 0.3309 - acc: 0.8614
Epoch 10/50
202/202 [==============================] - 0s 99us/step - loss: 0.3744 - acc: 0.8416
Epoch 11/50
202/202 [==============

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.5s remaining:    0.0s


Epoch 1/50
202/202 [==============================] - 9s 45ms/step - loss: 0.5057 - acc: 0.7277
Epoch 2/50
202/202 [==============================] - 0s 106us/step - loss: 0.3505 - acc: 0.8614
Epoch 3/50
202/202 [==============================] - 0s 110us/step - loss: 0.4234 - acc: 0.8119
Epoch 4/50
202/202 [==============================] - 0s 103us/step - loss: 0.3964 - acc: 0.8366
Epoch 5/50
202/202 [==============================] - 0s 99us/step - loss: 0.3416 - acc: 0.8663
Epoch 6/50
202/202 [==============================] - 0s 101us/step - loss: 0.4201 - acc: 0.8366
Epoch 7/50
202/202 [==============================] - 0s 98us/step - loss: 0.3495 - acc: 0.8564
Epoch 8/50
202/202 [==============================] - 0s 103us/step - loss: 0.3199 - acc: 0.8663
Epoch 9/50
202/202 [==============================] - 0s 102us/step - loss: 0.3038 - acc: 0.8812
Epoch 10/50
202/202 [==============================] - 0s 114us/step - loss: 0.3634 - acc: 0.8317
Epoch 11/50
202/202 [===========

...

Epoch 1/150
202/202 [==============================] - 10s 48ms/step - loss: 0.4734 - acc: 0.7822
Epoch 2/150
202/202 [==============================] - 0s 106us/step - loss: 0.3269 - acc: 0.8168
Epoch 3/150
202/202 [==============================] - 0s 100us/step - loss: 0.3024 - acc: 0.8168
Epoch 4/150
202/202 [==============================] - 0s 110us/step - loss: 0.2592 - acc: 0.8663
Epoch 5/150
202/202 [==============================] - 0s 99us/step - loss: 0.2490 - acc: 0.9158
Epoch 6/150
202/202 [==============================] - 0s 105us/step - loss: 0.2719 - acc: 0.9059
Epoch 7/150
202/202 [==============================] - 0s 109us/step - loss: 0.2540 - acc: 0.9109
Epoch 8/150
202/202 [==============================] - 0s 98us/step - loss: 0.2104 - acc: 0.9257
Epoch 9/150
202/202 [==============================] - 0s 106us/step - loss: 0.2057 - acc: 0.9257
Epoch 10/150
202/202 [==============================] - 0s 139us/step - loss: 0.1853 - acc: 0.9406
Epoch 11/150
202/202 

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.5min remaining:    0.0s


Epoch 1/200
202/202 [==============================] - 10s 48ms/step - loss: 0.5266 - acc: 0.7228
Epoch 2/200
202/202 [==============================] - 0s 105us/step - loss: 0.3833 - acc: 0.8267
Epoch 3/200
202/202 [==============================] - 0s 105us/step - loss: 0.3214 - acc: 0.8713
Epoch 4/200
202/202 [==============================] - 0s 103us/step - loss: 0.3236 - acc: 0.8564
Epoch 5/200
202/202 [==============================] - 0s 114us/step - loss: 0.3267 - acc: 0.8465
Epoch 6/200
202/202 [==============================] - 0s 107us/step - loss: 0.3138 - acc: 0.8515
Epoch 7/200
202/202 [==============================] - 0s 99us/step - loss: 0.2919 - acc: 0.8564
Epoch 8/200
202/202 [==============================] - 0s 100us/step - loss: 0.3202 - acc: 0.8663
Epoch 9/200
202/202 [==============================] - 0s 103us/step - loss: 0.3476 - acc: 0.8416
Epoch 10/200
202/202 [==============================] - 0s 109us/step - loss: 0.2664 - acc: 0.8861
Epoch 11/200
202/202

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  3.4min finished


Epoch 1/50
303/303 [==============================] - 10s 33ms/step - loss: 0.5371 - acc: 0.7030
Epoch 2/50
303/303 [==============================] - 0s 98us/step - loss: 0.3913 - acc: 0.8449
Epoch 3/50
303/303 [==============================] - 0s 92us/step - loss: 0.4361 - acc: 0.8317
Epoch 4/50
303/303 [==============================] - 0s 92us/step - loss: 0.3834 - acc: 0.8482
Epoch 5/50
303/303 [==============================] - 0s 90us/step - loss: 0.4079 - acc: 0.8152
Epoch 6/50
303/303 [==============================] - 0s 98us/step - loss: 0.3618 - acc: 0.8449
Epoch 7/50
303/303 [==============================] - 0s 94us/step - loss: 0.3492 - acc: 0.8614
Epoch 8/50
303/303 [==============================] - 0s 94us/step - loss: 0.4062 - acc: 0.8284
Epoch 9/50
303/303 [==============================] - 0s 93us/step - loss: 0.3503 - acc: 0.8647
Epoch 10/50
303/303 [==============================] - 0s 93us/step - loss: 0.3358 - acc: 0.8416
Epoch 11/50
303/303 [=================

**Hyperparameter tuning 6**

param_grid = {

    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50, 100, 150, 200],
    'kerasclassifier__learning_rate': [0.1],
    'kerasclassifier__schedule_decay': [0.01],
    'kerasclassifier__nodes': [8]
}

Output:

Cross Validation Score: 0.6468646864686468
Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50, 'kerasclassifier__learning_rate': 0.1, 'kerasclassifier__nodes': 8, 'kerasclassifier__schedule_decay': 0.01}

In [21]:
# There is a drop in accuracy so rerunning with different values

# Create pipeline
pipeline = make_pipeline(\
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [20, 50, 100, 150, 200, 500],
    'kerasclassifier__learning_rate': [0.1],
    'kerasclassifier__schedule_decay': [0.01],
    'kerasclassifier__nodes': [8]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=3, 
                         scoring='accuracy', verbose=10)

gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] kerasclassifier__batch_size=40, kerasclassifier__epochs=20, kerasclassifier__learning_rate=0.1, kerasclassifier__nodes=8, kerasclassifier__schedule_decay=0.01 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/20
202/202 [==============================] - 10s 52ms/step - loss: 0.4677 - acc: 0.7871
Epoch 2/20
202/202 [==============================] - 0s 102us/step - loss: 0.3380 - acc: 0.8465
Epoch 3/20
202/202 [==============================] - 0s 101us/step - loss: 0.3414 - acc: 0.8416
Epoch 4/20
202/202 [==============================] - 0s 104us/step - loss: 0.3200 - acc: 0.8564
Epoch 5/20
202/202 [==============================] - 0s 117us/step - loss: 0.3222 - acc: 0.8812
Epoch 6/20
202/202 [==============================] - 0s 120us/step - loss: 0.2709 - acc: 0.9010
Epoch 7/20
202/202 [==============================] - 0s 101us/step - loss: 0.3701 - acc: 0.8564
Epoch 8/20
202/202 [==============================] - 0s 104us/step - loss: 0.2757 - acc: 0.8861
Epoch 9/20
202/202 [==============================] - 0s 104us/step - loss: 0.2858 - acc: 0.8663
Epoch 10/20
202/202 [==============================] - 0s 112us/step - loss: 0.2507 - acc: 0.8762
Epoch 11/20
202/202 [========

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.6s remaining:    0.0s


Epoch 1/20
202/202 [==============================] - 10s 49ms/step - loss: 0.4729 - acc: 0.7624
Epoch 2/20
202/202 [==============================] - 0s 109us/step - loss: 0.3857 - acc: 0.8416
Epoch 3/20
202/202 [==============================] - 0s 107us/step - loss: 0.3378 - acc: 0.8614
Epoch 4/20
202/202 [==============================] - 0s 104us/step - loss: 0.3530 - acc: 0.8465
Epoch 5/20
202/202 [==============================] - 0s 120us/step - loss: 0.4072 - acc: 0.8168
Epoch 6/20
202/202 [==============================] - 0s 112us/step - loss: 0.3623 - acc: 0.8564
Epoch 7/20
202/202 [==============================] - 0s 104us/step - loss: 0.3235 - acc: 0.8762
Epoch 8/20
202/202 [==============================] - 0s 104us/step - loss: 0.3161 - acc: 0.8861
Epoch 9/20
202/202 [==============================] - 0s 100us/step - loss: 0.2822 - acc: 0.8960
Epoch 10/20
202/202 [==============================] - 0s 100us/step - loss: 0.4046 - acc: 0.8020
Epoch 11/20
202/202 [========

...

Epoch 1/100
202/202 [==============================] - 10s 52ms/step - loss: 0.5033 - acc: 0.7574
Epoch 2/100
202/202 [==============================] - 0s 108us/step - loss: 0.2958 - acc: 0.9010
Epoch 3/100
202/202 [==============================] - 0s 97us/step - loss: 0.2495 - acc: 0.8911
Epoch 4/100
202/202 [==============================] - 0s 99us/step - loss: 0.2044 - acc: 0.9109
Epoch 5/100
202/202 [==============================] - 0s 115us/step - loss: 0.2101 - acc: 0.9356
Epoch 6/100
202/202 [==============================] - 0s 121us/step - loss: 0.2115 - acc: 0.9307
Epoch 7/100
202/202 [==============================] - 0s 117us/step - loss: 0.2091 - acc: 0.9158
Epoch 8/100
202/202 [==============================] - 0s 114us/step - loss: 0.1909 - acc: 0.9307
Epoch 9/100
202/202 [==============================] - 0s 109us/step - loss: 0.2267 - acc: 0.9059
Epoch 10/100
202/202 [==============================] - 0s 107us/step - loss: 0.1716 - acc: 0.9208
Epoch 11/100
202/202 

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.5min remaining:    0.0s


Epoch 1/150
202/202 [==============================] - 11s 52ms/step - loss: 0.5644 - acc: 0.7376
Epoch 2/150
202/202 [==============================] - 0s 122us/step - loss: 0.3892 - acc: 0.8317
Epoch 3/150
202/202 [==============================] - 0s 109us/step - loss: 0.3268 - acc: 0.8663
Epoch 4/150
202/202 [==============================] - 0s 107us/step - loss: 0.3365 - acc: 0.8515
Epoch 5/150
202/202 [==============================] - 0s 105us/step - loss: 0.3328 - acc: 0.8317
Epoch 6/150
202/202 [==============================] - 0s 106us/step - loss: 0.3174 - acc: 0.8564
Epoch 7/150
202/202 [==============================] - 0s 126us/step - loss: 0.4137 - acc: 0.8020
Epoch 8/150
202/202 [==============================] - 0s 108us/step - loss: 0.3744 - acc: 0.8366
Epoch 9/150
202/202 [==============================] - 0s 107us/step - loss: 0.3079 - acc: 0.8515
Epoch 10/150
202/202 [==============================] - 0s 105us/step - loss: 0.3118 - acc: 0.8663
Epoch 11/150
202/20

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:  6.1min finished


Epoch 1/50
303/303 [==============================] - 11s 38ms/step - loss: 0.5999 - acc: 0.6667
Epoch 2/50
303/303 [==============================] - 0s 105us/step - loss: 0.3796 - acc: 0.8152
Epoch 3/50
303/303 [==============================] - 0s 108us/step - loss: 0.3720 - acc: 0.8449
Epoch 4/50
303/303 [==============================] - 0s 107us/step - loss: 0.3507 - acc: 0.8581
Epoch 5/50
303/303 [==============================] - 0s 105us/step - loss: 0.3264 - acc: 0.8680
Epoch 6/50
303/303 [==============================] - 0s 98us/step - loss: 0.3102 - acc: 0.8746
Epoch 7/50
303/303 [==============================] - 0s 107us/step - loss: 0.2945 - acc: 0.8878
Epoch 8/50
303/303 [==============================] - 0s 124us/step - loss: 0.3060 - acc: 0.8911
Epoch 9/50
303/303 [==============================] - 0s 112us/step - loss: 0.2803 - acc: 0.9010
Epoch 10/50
303/303 [==============================] - 0s 108us/step - loss: 0.2816 - acc: 0.8977
Epoch 11/50
303/303 [=========

Hyperparameter tuning 6

param_grid = {

'kerasclassifier__batch_size': [40],
'kerasclassifier__epochs': [50, 100, 150, 200],
'kerasclassifier__learning_rate': [0.1],
'kerasclassifier__schedule_decay': [0.01],
'kerasclassifier__nodes': [8]
}

Output:

Cross Validation Score: 0.7920792079207921
Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50, 'kerasclassifier__learning_rate': 0.1, 'kerasclassifier__nodes': 8, 'kerasclassifier__schedule_decay': 0.01}

**Final Output:**

Using optimizer Nadam

param_grid = {

    'kerasclassifier__batch_size': [40],
    'kerasclassifier__epochs': [50],
    'kerasclassifier__learning_rate': [0.1],
    'kerasclassifier__schedule_decay': [0.01],
    'kerasclassifier__nodes': [8]
}

Cross Validation Score: 0.7920792079207921 

Best Parameters: {'kerasclassifier__batch_size': 40, 'kerasclassifier__epochs': 50, 'kerasclassifier__learning_rate': 0.1, 'kerasclassifier__nodes': 8, 'kerasclassifier__schedule_decay': 0.01}